In [171]:
%run '05_Equivariant_Vector_Bundle.ipynb'

In [172]:
class Complex_Over_Irreducible_Homogeneous_Variety ( Algebraic_Structure ) :

    def __getitem__ ( self , Input:int ) -> "Object" :
        """Returns a single object."""
        if Input in ZZ :
            if Input in self.Objects().keys() : return self.Objects()[Input]
            else                              : return self.Zero_Object()

        else :
            raise ValueError('The variable ``Input`` is inappropriate.')


    def __iter__ ( self ) -> Iterator[ tuple[ int , "Object" ] ] :
        """Returns a generator for the objects of ``self``."""
        for Degree in sorted( self.Objects().keys() ) :
            Object = self[Degree]
            yield Degree , Object


    def __len__ ( self ) -> int :
        return len( self.Degrees() )


    def __lshift__ ( self , Shift:int==1 ) -> "Complex_Over_Irreducible_Homogeneous_Variety" :
        """Returns the left shift of ``self``."""
        return self >> -Shift


    def __repr__ ( self ) -> tuple[ Irreducible_Homogeneous_Variety , dict[ int : Equivariant_Vector_Bundle ] ]:
        """Returns all attributes which are necessary to initialize the object."""
        return self.Base_Space() , self.Objects()


    def __rshift__ ( self , Shift:int==1 ) -> "Complex_Over_Irreducible_Homogeneous_Variety" :
        """Returns the right shift of ``self``."""
        assert Shift in ZZ , \
               TypeError('The input for ``Shift`` need to be an integer.')
        return self.__class__( Base_Space=self.Base_Space() , Objects={ Degree+Shift : Object for Degree , Object in self } )


    def __str__ ( self ) -> str :
        """Returns a one-line string as short description."""
        Length = len(self)
        if   Length == 0 :
            return 'Zero complex over '+str(self.Base_Space())
        else :
            Previous_Degree = -infinity
            String = ''
            for Current_Degree , Object in self :
                if   Previous_Degree == -infinity :
                    String += '... --d_'+str(Current_Degree-2)+'--> 0 --d_'+str(Current_Degree-1)+'--> '+str(Object)+' --d_'+str(Current_Degree)+'--> '

                elif Previous_Degree <= Current_Degree-3 :
                    String += '... --d_'+str(Current_Degree-2)+'--> 0 --d_'+str(Current_Degree-1)+'--> '+str(Object)+' --'+str(Current_Degree)+'--> '

                elif Previous_Degree == Current_Degree-2 :
                    String += '0 --d_'+str(Current_Degree-1)+'--> '+str(Object)+' --d_'+str(Current_Degree)+'--> '

                elif Previous_Degree == Current_Degree-1 :
                    String += str(Object)+' --d_'+str(Current_Degree)+'--> '

                Previous_Degree = Current_Degree

            String += '0 --d_'+str(Current_Degree+1)+'--> ...'

            return String


    def Base_Space ( self ) -> Irreducible_Homogeneous_Variety :
        """Returns the attribute ``_Base_Space``."""
        return self._Base_Space


    def Degrees ( self ) -> list[ int ] :
        """Returns the keys of the dictionary ``Objects``."""
        return self.Objects().keys()


    def Objects ( self ) -> dict[ int : Equivariant_Vector_Bundle ] :
        """Returns the attribute ``_Objects``."""
        return self._Objects


In [212]:
class Complex_Of_Coherent_Sheaves ( Complex_Over_Irreducible_Homogeneous_Variety ) :

    def __call__ ( self , *Twists:tuple[int] ) -> "Complex_Of_Coherent_Sheaves" :
        """Returns the a twist of ``self``."""
        New_Objects = dict({})
        for Degree , Object in self :
            if type( Object ) in [ Irreducible_Equivariant_Vector_Bundle , Direct_Sum_Of_Equivariant_Vector_Bundles , Extension_Of_Equivariant_Vector_Bundles ] :
                New_Object = Object(*Twists)
            elif Object in [ 'Kernel' , 'Cokernel' ] :
                New_Object = Object
            New_Objects.update({ Degree : New_Object })
        return self.__class__( Base_Space=self.Base_Space() , Objects=New_Objects )


    def __init__ ( self , Base_Space:Irreducible_Homogeneous_Variety , Objects:dict ) -> None :
        assert isinstance( Base_Space , Irreducible_Homogeneous_Variety ) , \
               TypeError('The input for ``Base_Space`` need to be an irreducible homogeneous variety.')
        self._Base_Space = Base_Space

        if   isinstance( Objects , list ) : Objects = { Counter : Object for Counter , Object in enumerate( Objects ) }
        elif isinstance( Objects , dict ) : pass
        else                              : raise TypeError('The input for ``Objects`` need to be a list or a dictionary.')

        Zero = self.Zero_Object()
        self._Objects = dict({})
        for Degree , Object in Objects.items() :
            assert Degree in ZZ , \
                   TypeError('The degrees need to be integers.')

            if isinstance( Object , ( Irreducible_Equivariant_Vector_Bundle , Direct_Sum_Of_Equivariant_Vector_Bundles , Extension_Of_Equivariant_Vector_Bundles ) ) :
                assert Object.Base_Space() == Base_Space , \
                       TypeError('The base space of the object in degree '+str(Degree)+' does not coincide with the given base space.')
            elif Object in [ 'Kernel' , 'kernel' , 'Ker' , 'ker' ] :
                Object = 'Kernel'
            elif Object in [ 'Cokernel' , 'cokernel' , 'CoKernel' , 'coKernel' , 'Coker' , 'coker' , 'CoKer' , 'coKer' ] :
                Object = 'Cokernel'
            else :
                raise TypeError('The object in degree '+str(Degree)+' need to be an equivariant vector bundle.')

            if Object != Zero :
                self._Objects.update({ Degree : Object })

        # Test  d_i \circ d_i-1 = 0 numerically.
        self._Numerical_Decomposition_Of_Differential = dict({})
        self._Numerical_Cohomology = dict({})
        for Degree in self.Degrees() :
            for Current_Degree in [ Degree , Degree+1 ] :
                if not Current_Degree in self._Numerical_Cohomology.keys() :
                    Numerical_Kernel_In_Previous_Degree , Numerical_Image_In_Previous_Degree = self.Decompose_Differential_Numerically(Current_Degree-1)
                    Numerical_Kernel_In_Current_Degree  , Numerical_Image_In_Current_Degree  = self.Decompose_Differential_Numerically(Current_Degree)
                    if Numerical_Image_In_Previous_Degree in Numerical_Kernel_In_Current_Degree :
                        Numerical_Cohomology_Group = Numerical_Kernel_In_Current_Degree - Numerical_Image_In_Previous_Degree
                    else :
                        raise ValueError('Caution: d_'+str(Degree)+' circ d_'+str(Degree-1)+' = 0. \n' \
                                         + 'The numerical image of d_'+str(Degree-1)+' is '+str(Numerical_Image_In_Previous_Degree)+'. \n' \
                                         + 'The numerical kernel of d_'+str(Degree)+' is '+str(Numerical_Kernel_In_Current_Degree)+'.' \
                                        )
                    self._Numerical_Cohomology.update({ Current_Degree : Numerical_Cohomology_Group })


    def Numerical_Cohomology( self , *p:int ) -> dict[ int : Direct_Sum_Of_Equivariant_Vector_Bundles ]:
        """Returns the cohomology of ``self``."""
        if   len(p) == 0 : return self._Numerical_Cohomology

        elif len(p) == 1 :
            if p in self._Numerical_Cohomology.keys() : return { p : self._Numerical_Cohomology[p] }
            else : return dict({})

        else :
            raise ValueError('The input for ``p`` is of inappropriate length.')


    def Decompose_Differential_Numerically ( self , Degree:int ) -> tuple[ Direct_Sum_Of_Equivariant_Vector_Bundles ] :
        """Returns numerical kernel and numerical image of differential in given degree."""
        assert Degree in ZZ , \
               TypeError( 'The input for ``Degree`` need to be an integer.' )

        if Degree in self._Numerical_Decomposition_Of_Differential.keys() :
            Numerical_Kernel_In_Current_Degree , Numerical_Image_In_Current_Degree = self._Numerical_Decomposition_Of_Differential[Degree]
            return Numerical_Kernel_In_Current_Degree , Numerical_Image_In_Current_Degree

        else :
            Current_Object = self[Degree]
            if   type(Current_Object) in [ Irreducible_Equivariant_Vector_Bundle , Direct_Sum_Of_Equivariant_Vector_Bundles , Extension_Of_Equivariant_Vector_Bundles ] :
                if Current_Object == self.Zero_Object() :
                    Numerical_Kernel_In_Current_Degree = self.Zero_Object()
                    Numerical_Image_In_Current_Degree  = self.Zero_Object()
                else :
                    Domain = self.Zero_Object()
                    for Summand in Current_Object.Irreducible_Components() :
                        Domain += Summand
                    Numerical_Kernel_In_Subsequent_Degree , Numerical_Image_In_Subsequent_Degree = self.Decompose_Differential_Numerically( Degree+1 )
                    Numerical_Kernel_In_Current_Degree = -( ( Numerical_Kernel_In_Subsequent_Degree - Domain ).Negative_Part() )
                    Numerical_Image_In_Current_Degree  = Domain - Numerical_Kernel_In_Current_Degree

            elif Current_Object == 'Kernel'   :
                Numerical_Kernel_In_Subsequent_Degree , Numerical_Image_In_Subsequent_Degree = self.Decompose_Differential_Numerically( Degree+1 )
                Numerical_Kernel_In_Current_Degree = self.Zero_Object()
                Numerical_Image_In_Current_Degree  = Numerical_Kernel_In_Subsequent_Degree

            elif Current_Object == 'Cokernel' :
                PrePrevious_SemiSimplification = self.Zero_Object()
                for Summand in self[Degree-2].Irreducible_Components() :
                    PrePrevious_SemiSimplification += Summand
                Previous_SemiSimplification = self.Zero_Object()
                for Summand in self[Degree-1].Irreducible_Components() :
                    Previous_SemiSimplification += Summand

                Numerical_Kernel_In_Current_Degree = ( Previous_SemiSimplification - PrePrevious_SemiSimplification ).Positive_Part()
                Numerical_Image_In_Current_Degree  = self.Zero_Object()

            self._Numerical_Decomposition_Of_Differential.update({ Degree : ( Numerical_Kernel_In_Current_Degree , Numerical_Image_In_Current_Degree ) })
            return Numerical_Kernel_In_Current_Degree , Numerical_Image_In_Current_Degree


    def Dual ( self ) -> "Complex_Of_Coherent_Sheaves" :
        """Returns the a dual of ``self``."""
        New_Objects = dict({})
        for Degree , Object in self :
            if type( Object ) in [ Irreducible_Equivariant_Vector_Bundle , Direct_Sum_Of_Equivariant_Vector_Bundles , Extension_Of_Equivariant_Vector_Bundles ] :
                New_Object = Object.Dual()
            elif Object == 'Kernel'   : New_Object = 'Cokernel'
            elif Object == 'Cokernel' : New_Object = 'Kernel'
            New_Objects.update({ -Degree : New_Object })
        return self.__class__( Base_Space=self.Base_Space() , Objects=New_Objects )


    def Numerical_Decomposition ( self ) -> table :
        Rows = []
        for Degree in self.Degrees() :
            Numerical_Kernel , Numerical_Image = self.Decompose_Differential_Numerically(Degree)
            Block = [ [ self[Degree] ] , \
                      [ '|'] , \
                      [ '|' ] + [ 'Ker d_'+str(Degree)+':' ] + [ str(Summand) for Summand in Numerical_Kernel ] , \
                      [ '|'] , \
                      [ '| d_'+str(Degree) ] , \
                      [ '|'] , \
                      [ '|' ] + [ 'Im d_'+str(Degree)+':' ] + [ str(Summand) for Summand in Numerical_Image ] , \
                      [ 'V' ] \
                    ]
            Rows += Block

        Maximal_Row_Length = max( [ 0 ] + [ len(Row) for Row in Rows ] )
        Rows = [ Row + (Maximal_Row_Length-len(Row))*[ '' ] for Row in Rows ]

        return table( rows=Rows , header_column=True )


    def Is_Numerically_Exact ( self ) -> bool :
        Zero = self.Zero_Object()
        for Degree , Numerical_Cohomology_Group in self.Numerical_Cohomology().items() :
            if Numerical_Cohomology_Group != Zero : return False
        return True


    def Is_Short ( self ) -> bool :
        if len(self) == 3 and max(self.Degrees())-min(self.Degrees()) == 2 : return True
        else                                                               : return False


    def SemiSimplification ( self , Degree:int ) -> Direct_Sum_Of_Equivariant_Vector_Bundles :
        assert Degree in ZZ , \
               TypeError( 'The input for ``Degree`` need to be an integer.' )

        Numerical_Kernel , Numerical_Image = self.Decompose_Differential_Numerically(Degree)
        return Numerical_Kernel + Numerical_Image


    def Split_Into_Short_Sequences ( self ) :
        Zero = self.Zero_Object()
        Numerical_Cohomology = self.Numerical_Cohomology()
        Previous_Degree = -infinity
        for Degree in self.Degrees() :
            Numerical_Kernel_In_Current_Degree    , Numerical_Image_In_Current_Degree    = self.Decompose_Differential_Numerically(Degree)
            Numerical_Kernel_In_Subsequent_Degree , Numerical_Image_In_Subsequent_Degree = self.Decompose_Differential_Numerically(Degree+1)
            Short_Exact_Sequence = { 'Left'   : ( 'Ker d_'+str(Degree) , Numerical_Kernel_In_Current_Degree ) , \
                                     'Middle' : ( 'C_'+str(Degree)     , self[Degree]                       ) , \
                                     'Right'  : ( 'Im d_'+str(Degree)  , Numerical_Image_In_Current_Degree  )   \
                                   }
            Gluing_Datum         = { 'Left'   : ( 'Im d_'+str(Degree)      , Numerical_Image_In_Current_Degree ) , \
                                     'Middle' : ( 'Ker d_'+str(Degree+1)   , Numerical_Kernel_In_Subsequent_Degree  ) , \
                                     'Right'  : ( 'H^'+str(Degree+1)+'(C)' , self.Numerical_Cohomology()[Degree+1]  )   \
                                   }
            yield Degree , Short_Exact_Sequence , Gluing_Datum


    def Zero_Object ( self ) :
        return self.Base_Space().Zero_Vector_Bundle()


    def Apply ( self , Other , Direction:str ) :
        if   Direction == 'From' : Variance = 'Co'
        elif Direction == 'Into' : Variance = 'Contra'
        else : raise ValueError('The input for ``Direction`` is inappropriate.')

        for Differential_Degree , Short_Exact_Sequence , Gluing_Datum in self.Split_Into_Short_Sequences() :
            print( 'Consider the differential d_'+str(Differential_Degree)+':' )
            for Sequence_Label , Sequence_Data in [ ( 'Constituent part' , Short_Exact_Sequence ) , ( 'Gluing datum' , Gluing_Datum ) ] :
                print( '('+Sequence_Label+')' )

                if Sequence_Label == 'Gluing datum' and Sequence_Data['Right'][1] == self.Zero_Object() :
                    print( Sequence_Data['Left'][0] , '=' , Sequence_Data['Middle'][0] )

                else :
                    LES = Long_Exact_Sequence_Of_EXT_Spaces( self.Base_Space() , Other , Sequence_Data , Variance=Variance )
                    Rows = []
                    for Cohomology_Degree , Result in LES.Compute() :
                        Row1 = [ 'Degree = '+str(Cohomology_Degree)+' :' ] + [ '...' , '-->' ]
                        Row2 = [ ''                                      ] + [ ''    , ''    ]
                        for Position in LES.POSITIONS :
                            if Result[Position][1] == None : Value = ''
                            else                           : Value = '= '+str(Result[Position][1])
                            Row1 += [ Result[Position][0] , '-->' ]
                            Row2 += [ Value               , ''    ]
                        Row1 += [ '...' ]
                        Row2 += [ ''    ]
                        Rows += [ Row1 , Row2 ]
                    print( table( rows=Rows ) )
            print()


In [174]:
class Long_Exact_Sequence_Of_EXT_Spaces ( Algebraic_Structure ) :

    POSITIONS = [ 'Left' , 'Middle' , 'Right' ]

    def __init__ ( self , \
                   Base_Space:Irreducible_Homogeneous_Variety , \
                   Other , \
                   Short_Exact_Sequence , \
                   Variance \
                 ) -> None :
        # Base space
        assert type(Base_Space) == Irreducible_Homogeneous_Variety , \
            TypeError('The input for ``Base_Space`` need to be an irreducible homogeneous variety.')
        self._Base_Space = Base_Space

        # Variance
        assert Variance in [ 'Co' , 'Contra' ] , \
            ValueError('The variance need to be `co` or `Contra`.')
        self._Variance = Variance

        # Other
        if   type(Other) == tuple :
            assert len(Other) == 2 , \
                ValueError('If the input for ``other`` is a tuple, then it need to be of length 2.')
            Other_Label , Other_Object = Other
            assert type(Other_Label) == str , \
                   TypeError('If the input for ``Other`` is a 2-tuple, then the first entry need to be a string.')
            assert type(Other_Object) in [ Irreducible_Equivariant_Vector_Bundle , Direct_Sum_Of_Equivariant_Vector_Bundles , Extension_Of_Equivariant_Vector_Bundles ] , \
                   TypeError('If the input for ``Other`` is a 2-tuple, then the second entry need to be a equivariant vector bundle.')

        elif type(Other) in [ Irreducible_Equivariant_Vector_Bundle , Direct_Sum_Of_Equivariant_Vector_Bundles , Extension_Of_Equivariant_Vector_Bundles ] :
            if   self.Variance() == 'Co'     : Other_Label = 'M'
            elif self.Variance() == 'Contra' : Other_Label = 'N'

        else :
            raise ValueError('The input for ``Other`` is inappropriate.')

        self._Other = ( Other_Label , Other_Object )

        # Short exact sequence
        if   type(Short_Exact_Sequence) == tuple :
            assert len(Short_Exact_Sequence) == 3 , \
                ValueError('If the input for ``Short_Exact_Sequence`` is a tuple, then it need to be of length 3.')
            Left , Middle , Right = Short_Exact_Sequence
            self._Short_Exact_Sequence = { 'Left' : Left , 'Middle' : Middle , 'Right' : Right }

        elif type(Short_Exact_Sequence) == dict :
            assert len(Short_Exact_Sequence.keys()) == 3 and set(Short_Exact_Sequence.keys()) == set(self.POSITIONS) , \
                ValueError('If the input for ``Short_Exact_Sequence`` is a dictionary, then the keys need to be the list '+str(self.POSITIONS)+'.')
            self._Short_Exact_Sequence = Short_Exact_Sequence

        else :
            raise ValueError('The input for ``Short_Exact_Sequence`` is inappropriate.')


    def __repr__ ( self ) :
        """Returns all attributes which are necessary to initialize the object."""
        return self.Base_Space() , self.Other() , self.Short_Exact_Sequence() , self.Variance()


    def __str__ ( self ) -> str :
        """Returns a one-line string as short description."""
        Other_Label  , Other_Object  = self.Other()
        Left_Label   , Left_Object   = self.Left()
        Middle_Label , Middle_Object = self.Middle()
        Right_Label  , Right_Object  = self.Right()

        if   self.Variance() == 'Co' :
            String = '... --> EXT^i( '+Other_Label+' , '+Left_Label+' ) --> EXT^i( '+Other_Label+' , '+Middle_Label+' ) --> EXT^i( '+Other_Label+' , '+Right_Label+' ) --> ... \n' + \
                     'where \n ' + \
                     str( table( Rows=[ [ Other_Label  , '=' , Other_Object  ] , \
                                        [ Left_Label   , '=' , Left_Object   ] , \
                                        [ Middle_Label , '=' , Middle_Object ] , \
                                        [ Right_Label  , '=' , Right_Object  ]   \
                                      ]
                               )
                        )

        elif self.Variance() == 'Contra' :
            String = '... --> EXT^i( '+Right_Label+' , '+Other_Label+' ) --> EXT^i( '+Middle_Label+' , '+Other_Label+' ) --> EXT^i( '+Left_Label+' , '+Other_Label+' ) --> ... \n' + \
                     'where \n ' + \
                     str( table( Rows=[ [ Right_Label  , '=' , Right_Object  ] , \
                                        [ Middle_Label , '=' , Middle_Object ] , \
                                        [ Left_Label   , '=' , Left_Object   ] , \
                                        [ Other_Label  , '=' , Other_Object  ]   \
                                      ]
                               )
                        )

        else :
            String = "None."

        return String


    def Base_Space ( self ) -> Irreducible_Homogeneous_Variety :
        """Returns the attribute ``_Base_Space``."""
        return self._Base_Space


    def Compute ( self ) :
        WCR = self.Base_Space().Parent_Group().Weyl_Character_Ring()

        Other_Label , Other_Object = self.Other()

        EXT = dict({})
        if   self.Variance() == 'Co' :
            for Position , ( Label , Object ) in self.Short_Exact_Sequence().items() :
                Result_Label = '( '+Other_Label+' , '+Label+' )'
                if   type(Object) in [ Irreducible_Equivariant_Vector_Bundle , Direct_Sum_Of_Equivariant_Vector_Bundles , Extension_Of_Equivariant_Vector_Bundles ]:
                    Result_Value = Other_Object.EXT( Object )
                else :
                    Result_Value = None
                EXT.update({ Position : ( Result_Label , Result_Value ) })

        elif self.Variance() == 'Contra' :
            for Position , ( Label , Object ) in self.Short_Exact_Sequence().items() :
                Result_Label = '( '+Label+' , '+Other_Label+' )'
                if   type(Object) in [ Irreducible_Equivariant_Vector_Bundle , Direct_Sum_Of_Equivariant_Vector_Bundles , Extension_Of_Equivariant_Vector_Bundles ]:
                    Result_Value = Object.EXT( Other_Object )
                else :
                    Result_Value = None
                if   Position == 'Left'   : EXT.update({ 'Right'  : ( Result_Label , Result_Value ) })
                elif Position == 'Middle' : EXT.update({ 'Middle' : ( Result_Label , Result_Value ) })
                elif Position == 'Right'  : EXT.update({ 'Left'   : ( Result_Label , Result_Value ) })

        Degrees = set({})
        for Position , ( Result_Label , Result_Value ) in EXT.items() :
            if type(Result_Value) == dict : Degrees = Degrees.union( set( Result_Value.keys()   ) )
        Degrees = sorted( list(Degrees) )

        if len(Degrees) == 0 :
            yield 'i' , { Position : ( 'EXT^i'+EXT[Position][0] , EXT[Position][1] ) for Position in self.POSITIONS }
        else :
            for Degree in Degrees :
                Result = dict({})
                for Position in self.POSITIONS :
                    Result_Label , Result_Value = EXT[Position]
                    if type(Result_Value) == dict :
                        if Degree in Result_Value.keys() : Result_Value = Result_Value[Degree]
                        else                             : Result_Value = WCR(0)
                    Result.update({ Position : ( 'EXT^'+str(Degree)+Result_Label , Result_Value ) })

                yield Degree , Result


    def Short_Exact_Sequence ( self ) :
        """Returns the attribute ``_Short_Exact_Sequence``."""
        return self._Short_Exact_Sequence


    def Other ( self ) :
        """Returns the attribute ``_Other``."""
        return self._Other


    def Variance ( self ) :
        """Returns the attribute ``_Variance``."""
        return self._Variance